In [1]:
import numpy as np
import pylab as pp
import pandas as pd
import os

pp.style.use("bmh")

os.chdir("D:\Stocks\PersianCal")
import PersianCal

In [2]:
filename = "SekeJadid_D_COM - Copy.csv"
loc = "D:\Stocks\Data"

os.chdir(loc)

# Loading in the Data
df = pd.read_csv(filename, delimiter=",")

In [3]:
## Datetime conversion
df['<DTYYYYMMDD>'] = pd.to_datetime(df['<DTYYYYMMDD>'], format='%Y%m%d')

# Dropping any NaNs
df.dropna(inplace=True)

df.rename(columns={"<DTYYYYMMDD>": "Date"}, inplace=True)

In [4]:
d = []
for i in range(len(df["Date"])):
    d.append(PersianCal.Gregorian(str(df["Date"][i])[0:10]).persian_string()) #Tabdil tarikh be Irani

In [5]:
for i in range(len(df["Date"])):
    df["Date"] = df["Date"].replace(df["Date"][i], d[i])

In [6]:
date_list = []
for date in df["Date"]:
    date_list.append(date[:7])
    
Dates = df["Date"].tolist() #kole tarikh ha
myDates = list(dict.fromkeys(date_list)) #Eshterake Tatikh ha

In [7]:
ext = np.zeros((len(myDates), 2)) #[[max], [min]]
ind = np.zeros((len(myDates), 1)).tolist()

#first indicator
for date in Dates:
    if (date[:7] in myDates[0]):
        ind1 = Dates.index(date) #یافتن مکان آخرین روز ماه

ind[0] = ind1

# First max and min:

In [8]:
a = [[], [], [], []]
names = ["<Open>", "<High>", "<Low>", "<Close>"]

for i in range(4):
    for j in names:
        a[i] = df[j][0+1:ind1+1].tolist()

ext[0][0] = max(max(a))
ext[0][1] = min(min(a))

# Calculating all of the extremums:

In [9]:
for i in range(1, len(myDates)):
    for date in Dates:
        if (date[:7] in myDates[i]):
            ind1 = Dates.index(date)
    ind[i] = ind1
    a = [[], [], [], []]
    for ii in range(4):
        for j in names:
            a[ii] = df[j][ind[i-1]+1:ind[i]+1].tolist()
    ext[i][0] = max(max(a))
    ext[i][1] = min(min(a))

In [10]:
Data = {"Date": myDates,
        "Max": ext[:, 0],
        "Min": ext[:, 1]}
DataF = pd.DataFrame(Data)
DataF.set_index('Date', inplace=True)

DataF.to_csv("output.csv")

In [12]:
# %matplotlib notebook
# import pylab as pp

# pp.plot(DataF["Date"], DataF["Max"], color='skyblue', label = "Max")
# pp.plot(DataF["Date"], DataF["Min"], color='red', label = "Min")
# pp.legend();

In [15]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [207]:
# listOfPositions = getIndexes(df[ind[4]+1:ind[5]+1], ext[5][1])
# print('Index positions of 1835000 in Dataframe: ')
# for i in range(len(listOfPositions)):
#     print('Position ', i, ' (Row index , Column Name) : ', listOfPositions[i])

In [203]:
minDate = []
firstPos = getIndexes(df[0+1:ind[0]], ext[0][0])
minDate.append(df.Date[firstPos[0][0]])

for j in range(len(ext)-1):
    listOfPositions = getIndexes(df[ind[j]+1:ind[j+1]+1], ext[j+1][1])
    for i in range(len(listOfPositions)):
        minDate.append(df.Date[listOfPositions[0][0]])

minDate = list(dict.fromkeys(minDate))

In [208]:
maxDate = []
firstPos = getIndexes(df[0+1:ind[0]], ext[0][0])
maxDate.append(df.Date[firstPos[0][0]])

for j in range(len(ext)-1):
    listOfPositions = getIndexes(df[ind[j]+1:ind[j+1]+1], ext[j+1][0])
    for i in range(len(listOfPositions)):
        maxDate.append(df.Date[listOfPositions[0][0]])
        
maxDate = list(dict.fromkeys(maxDate))

In [239]:
maxD = {"Date": maxDate,
        "Max": ext[:, 0]}

maxDF = pd.DataFrame(maxD)

In [240]:
minD = {"Date": minDate,
        "Min": ext[:, 1]}

minDF = pd.DataFrame(minD)

In [241]:
MM = {"Max Date": maxDate,
      "Max": ext[:, 0],
      "Min Date": minDate,
      "Min": ext[:, 1]}

MMF = pd.DataFrame(MM)
MMF


In [272]:
# first = np.zeros((len(maxDate), 1))
# last = np.zeros((len(maxDate), 1))

first = []
last = []

for i in range(len(maxDate)):
    if (int(maxDate[1][8:10]) < int(minDate[1][8:10])):
        first.append(ext[i, 0])
        last.append(ext[i, 1])
    else:
        first.append(ext[i, 1])
        last.append(ext[i, 0])

In [273]:
res = {"Date": myDates,
       "First": first,
       "Last": last}
resF = pd.DataFrame(res)
resF

,Date,First,Last
0,1386-09,1830000.0,1840000.0
1,1386-10,1855000.0,1990000.0
2,1386-11,1975000.0,2165000.0
3,1386-12,2160000.0,2330000.0
4,1387-01,2070000.0,2320000.0
...,...,...,...
145,1398-10,45110000.0,51310000.0
146,1398-11,48450000.0,53060000.0
147,1398-12,53300000.0,61410000.0
148,1399-01,61270000.0,65330000.0
